In [4]:
import numpy as np
import matplotlib.pyplot as pl
import sklearn
import pandas as pd

Построить временной ряд с суммами покупки

Построить временной ряд где мы смотрим по промежуток между покупками

Интерполяция функции по временному ряду


In [7]:
data = pd.read_parquet(r"D:\MyFolder\2.Work\DigitalBreaktrough\cifrovoi_proriv_hakaton\data\raw\train_dataset_hackaton2023_train.gzip")
data.head()

,customer_id,date_diff_post,buy_post,group_name,revenue,startdatetime,dish_name,ownareaall_sqm,format_name
0,29891,9.0,1,train,69.99,2022-12-05 12:03:58,Кинг Фри станд,300.0,Отдельно стоящий без внешней зоны
1,29891,9.0,1,train,190.00,2022-12-05 12:03:58,Чикен Тар-Тар,300.0,Отдельно стоящий без внешней зоны
2,29891,9.0,1,train,9.99,2022-12-05 12:03:58,Соус Сырный,300.0,Отдельно стоящий без внешней зоны
3,29891,9.0,1,train,119.99,2022-12-05 12:03:58,Энергет.нап. Адреналин Раш,300.0,Отдельно стоящий без внешней зоны
4,29891,9.0,1,train,119.99,2022-12-05 14:28:35,Латте (СТАНД.),300.0,Отдельно стоящий без внешней зоны


In [60]:
data_sorted = data.sort_values(by=['customer_id', 'startdatetime'])

# Добавление столбца с разницей времени
data_sorted['time_diff'] = data_sorted.groupby('customer_id')['startdatetime'].diff()

# Идентификация новых заказов
threshold1 = pd.Timedelta('1 day')  # Пороговое значение, можно изменить по необходимости
data_sorted['new_order'] = data_sorted['time_diff'] > threshold1

# Преобразование разницы во времени в количество дней (float)
data_sorted['time_diff_days'] = data_sorted['time_diff'].dt.days.astype(float)

# Группировка и сбор результатов
result = data_sorted.groupby('customer_id')['time_diff_days'].agg(list)

In [61]:
result

customer_id
29891       [nan, 0.0, 0.0, 0.0, 0.0, 9.0, 0.0, 5.0, 1.0, ...
30477       [nan, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
31426       [nan, 0.0, 0.0, 6.0, 0.0, 0.0, 4.0, 0.0, 0.0, ...
44491       [nan, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 26.0, 0.0,...
44939       [nan, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
                                  ...                        
46624509    [nan, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...
46625265    [nan, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
46639170    [nan, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
46654016                       [nan, 0.0, 0.0, 0.0, 0.0, 0.0]
46661804    [nan, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Name: time_diff_days, Length: 500000, dtype: object

In [65]:
result_modified = result.apply(lambda x: x[1:] if len(x) > 1 else x)
result_modified

customer_id
29891       [0.0, 0.0, 0.0, 0.0, 9.0, 0.0, 5.0, 1.0, 0.0, ...
30477       [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, ...
31426       [0.0, 0.0, 6.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, ...
44491          [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 26.0, 0.0, 0.0]
44939       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, ...
                                  ...                        
46624509    [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...
46625265    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
46639170    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
46654016                            [0.0, 0.0, 0.0, 0.0, 0.0]
46661804    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Name: time_diff_days, Length: 500000, dtype: object

In [66]:
result_modified[46661804]

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [71]:
result_modified = result_modified.apply(lambda x: [item for item in x if item != 0] if x else x)
result_modified.tail(10)

customer_id
46598535       []
46599400       []
46606537    [1.0]
46609455    [1.0]
46617009       []
46624509    [1.0]
46625265       []
46639170       []
46654016       []
46661804       []
Name: time_diff_days, dtype: object

## Изучение возвращаемости клиентов

In [69]:
data[data["customer_id"]==46661804]

,customer_id,date_diff_post,buy_post,group_name,revenue,startdatetime,dish_name,ownareaall_sqm,format_name
24261964,46661804,NaN,0,train,89.99,2023-08-01 17:50:26,Двойной Чизбургер,300.0,Фудкорт с туалетом
24261965,46661804,NaN,0,train,70.00,2023-08-01 17:50:26,Кинг Фри станд,300.0,Фудкорт с туалетом
24261966,46661804,NaN,0,train,100.00,2023-08-01 17:50:26,"Фрустайл Лимон Лайм мал 0,4",300.0,Фудкорт с туалетом
24261967,46661804,NaN,0,train,17.99,2023-08-01 18:01:40,Кинг Фри мал,300.0,Фудкорт с туалетом
24261968,46661804,NaN,0,train,79.98,2023-08-01 18:01:40,Чизбургер,300.0,Фудкорт с туалетом
24261969,46661804,NaN,0,train,89.98,2023-08-01 18:01:40,Сандэй Шоколадный,300.0,Фудкорт с туалетом
24261970,46661804,NaN,0,train,70.03,2023-08-01 18:01:40,Наггетсы (3 шт.),300.0,Фудкорт с туалетом
24261971,46661804,NaN,0,train,7.99,2023-08-01 18:01:40,"Липтон Лимон Дж 0,3",300.0,Фудкорт с туалетом
24261972,46661804,NaN,0,train,19.00,2023-08-01 18:01:40,Сырные Медальоны (3 шт.),300.0,Фудкорт с туалетом
24261973,46661804,NaN,0,train,89.98,2023-08-01 18:04:56,Сандэй Шоколадный,300.0,Фудкорт с туалетом


In [76]:
result_sorted = result_modified.apply(len).sort_values(ascending=False)
result_sorted

customer_id
36054024    33
292143      32
12615056    32
19168730    31
218116      31
            ..
39238540     0
2690962      0
19538974     0
19541704     0
46661804     0
Name: time_diff_days, Length: 500000, dtype: int64

In [114]:
result_zero = result_sorted[result_sorted < 10]

customer_id
36241555    15
18229790    15
11982017    15
17797572    15
36243756    15
            ..
39238540     0
2690962      0
19538974     0
19541704     0
46661804     0
Name: time_diff_days, Length: 494505, dtype: int64

In [86]:
result_zero

customer_id
21094629    9
37914379    9
24068114    9
14386430    9
16916607    9
           ..
39238540    0
2690962     0
19538974    0
19541704    0
46661804    0
Name: time_diff_days, Length: 475495, dtype: int64

In [116]:
100 - len(result_sorted[result_sorted < 16])/5000

1.0990000000000038

In [105]:
result_df = pd.DataFrame({'time_series': result_modified, 'quant_od_days': result_sorted})